Here we implement the solver from [Quantum algorithm for solving linear differential equations: Theory and experiment](https://journals.aps.org/pra/pdf/10.1103/PhysRevA.101.032307) by Xin *et al.* in the **Classiq** quantum programming language. 

### Classical Problem

Xin *et al.* provide a new method to solve any Linear Differential Equation formulated as 

$$\frac{d}{dt}\vec{x}(t)= M\vec{x}(t)+\vec{b}$$

for an $N\times N$ matrix $M$ and $N$-dimensional vectors $x$, $b$. 

Taylor-expanding the analytical solution, 

$$
\begin{aligned}
x(t)&=e^{Mt}x(0)+\left(e^{Mt}-\mathbb{I}\right)M^{-1}b
\\
&\approx \sum^k_{m=0}\frac{(Mt)^m}{m!}x(0)+\sum^k_{n=1}\frac{M^{n-1}t^n}{n!}b.
\end{aligned}
$$

### Quantum Formulation

1. Prepare two quantum states $\ket{x(0)}$ and $\ket{b}$ in the $N$-dimensional computational basis, on $\log_2(N)$ qubits. 
1. Describe the $N\times N$ operator $M$ as $A$ in the computational basis, such that $A_{ij}=M_{ij}\ket{i}\bra{j}$ up to normalization.

We then arrive at 

$$\ket{x(t)}\approx \sum_{m=0}^k \frac{\|x(0)\|(\|M\|At)^m}{m!}\ket{x(0)}+\sum_{n=1}^k\frac{\|b\|(\|M\|A)^{n-1}t^n}{n!}\ket{b}$$



1. Assuming $A$ is unitary, encode $k$ powers of $A$ as the unitaries $U_n=A^n$.
1. Define $C_m=\|x(0)\|(\|M\|t)^m/m!$ and $D_n=\|b\|(\|M\|t)^{n-1}t/n!$



In [ ]:
from classiq import (
    qfunc, Output, QBit, QNum, QArray, CInt,
    allocate, within_apply, control, repeat, if_, 
    inplace_prepare_state, hadamard_transform, inplace_prepare_int, 
    create_model, synthesize, execute, write_qmod, show,
    X, H, SWAP, 
    set_constraints, Constraints, set_execution_preferences, set_preferences, Preferences, QuantumProgram, 
)

